In [9]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch
from sklearn.metrics import f1_score

load_dotenv


ModuleNotFoundError: No module named 'ray.tune.suggest'

In [2]:
# Assuming feature_df and targets_df are already defined
data_dir = os.getenv("DATA")
minmax_df = pd.read_csv(Path(data_dir) / "minmax_dataset.csv")
targets_df = pd.read_csv(Path(data_dir) / "target.csv")

In [3]:
X = minmax_df.sample(n=50000)
ids = X.id
y = targets_df[targets_df.id.isin(ids)].drop(columns=["id"]).values.reshape(-1)
X = X.values

In [4]:
X_id = ray.put(X)
y_id = ray.put(y)

2024-09-30 12:18:44,996	WARNING services.py:2022 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67096576 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-09-30 12:18:45,089	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [5]:
ray.available_resources()

{'GPU': 1.0,
 'memory': 23334279373.0,
 'accelerator_type:G': 1.0,
 'object_store_memory': 10000000000.0,
 'CPU': 12.0,
 'node:__internal_head__': 1.0,
 'node:172.17.0.2': 1.0}

In [6]:
from utils.evaluators import evaluate_network

In [7]:
def train_dnn(config):
    X = ray.get(X_id)
    y = ray.get(y_id)
    _, _, f1 = evaluate_network(X, y, lr=config["learning_rate"], layer_sizes=[config["units_1"], config["units_1"]])
    return {"f1_score": f1}


def train_model(config):
    return train_dnn(config)


search_space = {
    # DNN hyperparameters
    "units_1": tune.randint(16, 128),
    "units_2": tune.randint(16, 128),
    "learning_rate": tune.loguniform(1e-4, 1e-2)
}

ray.init(num_cpus=8, num_gpus=1, ignore_reinit_error=True)

analysis = tune.run(train_model,
                    config=search_space,
                    num_samples=50,
                    search_alg=BayesOptSearch(metric="f1_score", mode="max"),
                    scheduler=ASHAScheduler(metric="f1_score", mode="max"),
                    resources_per_trial={
                        "cpu": 1,
                        "gpu": 1/8
                    })

best_config = analysis.get_best_config(metric="f1_score", mode="max")
best_trial = analysis.get_best_trial(metric="f1_score", mode="max")

print("Best config:", best_config)
print("Best F1 Score:", best_trial.last_result["f1_score"])

ray.shutdown()

2024-09-30 12:18:46,922	INFO worker.py:1619 -- Calling ray.init() again after it has already been called.
2024-09-30 12:18:46,924	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(train_model pid=46029) 
(train_model pid=46023) 
(train_model pid=46028) 
(train_model pid=46024) 
(train_model pid=46022) 
(train_model pid=46026) 
(train_model pid=46027) 
(train_model pid=46025) 
(train_model pid=46029) Loss: 15266.570238834234
(train_model pid=46029) Loss: 1160.1594111709855 [repeated 16x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(train_model pid=46023) Loss: 19834.02746124824 [repeated 14x across cluster]
(train_model pid=46029) Loss: 1098.2790231795516 [repeated 14x across cluster]
(train_model pid=46023) Loss: 776.2787184864283 [repeated 13x across cluster]
(train_model pid=46026) Loss: 28884.76565052433 [repeated 14x across cluster]
(train_model pid=46023) Loss: 795.2306010648608 [repeated 13x across cluster]
(train_model pid=46029) Loss: 1081.4055783916265 [repeated 16x acros

Trial name,f1_score
train_model_247ab_00000,0
train_model_247ab_00001,0
train_model_247ab_00002,0
train_model_247ab_00003,0
train_model_247ab_00004,0
train_model_247ab_00005,0
train_model_247ab_00006,0.0741847
train_model_247ab_00007,0.000969932
train_model_247ab_00008,0.0741452
train_model_247ab_00009,0


2024-09-30 12:37:09,803	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_2024-09-30_12-18-46' in 0.0112s.
2024-09-30 12:37:09,824	INFO tune.py:1041 -- Total run time: 1102.90 seconds (1102.85 seconds for the tuning loop).


Best config: {'units_1': 127, 'units_2': 118, 'learning_rate': 0.00018706814905236274}
Best F1 Score: 0.07578417291699914
(train_model pid=51378) Loss: 26110.36237318618 [repeated 2x across cluster]


In [ ]:
ray.available.res